In [1]:
import shutil
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [2]:
from IPython.core.display import clear_output

### DOWNLOAD PHOTO

In [2]:
url = 'https://c1.staticflickr.com/9/8094/8534020390_ee8b1b9e67_h.jpg'
dest_folder = 'data/images/'
file_name = 'test.jpg'
path_name = dest_folder + file_name
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)
r = requests.get(url, stream=True)
with open(path_name, 'wb') as out_file:
    shutil.copyfileobj(r.raw, out_file)
del r

### SEARACH PHOTOS

In [51]:
landscape_search_url = 'https://www.flickr.com/search/'
landscape_search_kwds = {'dimension_search_mode': 'min&height=1024&width=1024',
                         'media': 'photos',
                         'advanced': '1',
                         'text': 'landscape pro'}
headers = {'user-agent': 'Mozilla/5.0'}
r = requests.get(landscape_search_url, params=landscape_search_kwds, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [52]:
soup.select('div.content')

[]

In [53]:
soup.find_all('div', class_='photo-list-photo-interaction')

[]

### SEARCH PEOPLE

In [57]:
people_search_url = 'https://www.flickr.com/photos/12037949754@N01/'
# people_search_kwds = {'username': 'pro'}
headers = {'user-agent': 'Mozilla/5.0'}
r = requests.get(landscape_search_url)
soup = BeautifulSoup(r.text, 'html.parser')

### FLICKR API PHOTO SEARCH

In [10]:
keys_list = []
keys = {}
with open('flickr.keys') as f_open:
    for line in f_open:
        keys_list.append(line.strip('\n'))

In [14]:
for key in keys_list:
    pair = key.split(":")
    keys[pair[0]] = pair[1]

In [15]:
keys

{'Key': '933b26674bf1f7a00109f2f8eda30b52', 'Secret': '1bd86a22b5ea95bf'}

In [127]:
def search_flickr(search_tag_term, page_num=1):
    search_url = 'https://api.flickr.com/services/rest/'
    search_kwds = {'method':'flickr.photos.search',
                   'api_key':keys['Key'],
                   'tags':search_tag_term,
                   'sort':'relavance',
                   'content_type':'1',
                   'per_page':'500',
                   'page':page_num,
                   'extras': 'license, date_upload, date_taken, owner_name, original_format, last_update, geo, tags, machine_tags, o_dims, views, media, path_alias, url_o'
                  }
    r = requests.get(search_url, params=search_kwds)
    soup = BeautifulSoup(r.text, 'lxml')
    return soup, r.status_code

In [128]:
search_tag_term = 'landscape'
soup, status= search_flickr(search_tag_term)
total_pages = int(soup.photos.get('pages'))
print 'Status: {}'.format(status)

Status: 200


In [129]:
picture_soup_data = []
for i in range(1, total_pages+1):
    soup, status = search_flickr(search_tag_term, page_num=i)
    if int(status) != 200:
        print "Error on Page {}: Status Code {}".format(i, status)
        print "Print soup to determine problem."
        break
    if i == 1:
        print "Begining Search for '{}': {} Pages Found".format(search_tag_term, total_pages)
    picture_soup_data.extend(soup.findAll('photo'))
    if i%50==0:
        print "{} Pages Complete".format(i)
    if i == total_pages:
        print "Search Complete"
        print "-"*30

Begining Search for 'landscape': 980 Pages Found
50 Pages Complete
100 Pages Complete
150 Pages Complete
200 Pages Complete
250 Pages Complete
300 Pages Complete
350 Pages Complete
400 Pages Complete
450 Pages Complete
500 Pages Complete
550 Pages Complete
600 Pages Complete
650 Pages Complete
700 Pages Complete
750 Pages Complete
800 Pages Complete
850 Pages Complete
900 Pages Complete
950 Pages Complete
Search Complete
------------------------------


In [130]:
len(picture_soup_data)

489963

In [132]:
a = picture_soup_data[0]

In [134]:
a.attrs.keys()

['pathalias',
 'ispublic',
 'media_status',
 'originalformat',
 'owner',
 'id',
 'lastupdate',
 'datetakengranularity',
 'media',
 'views',
 'datetaken',
 'farm',
 'secret',
 'ownername',
 'latitude',
 'accuracy',
 'isfamily',
 'machine_tags',
 'tags',
 'isfriend',
 'dateupload',
 'width_o',
 'originalsecret',
 'license',
 'datetakenunknown',
 'height_o',
 'longitude',
 'server',
 'url_o',
 'context',
 'title']

In [107]:
columns = ['id', 'title', 'owner', 'secret', 'farm', 'server', 'ispublic']
int_columns = ['id', 'farm', 'server', 'ispublic']

In [112]:
picture_data = []
for i, picture in enumerate(pictures):
    picture_data.append([int(picture.get(name)) if name in int_columns else picture.get(name) for name in columns])
    if i%50000==0:
        print float(i)/len(pictures)*100," percent complete"

0.0  percent complete
10.0813565473  percent complete
20.1627130947  percent complete
30.244069642  percent complete
40.3254261893  percent complete
50.4067827367  percent complete
60.488139284  percent complete
70.5694958314  percent complete
80.6508523787  percent complete
90.732208926  percent complete


In [114]:
df = pd.DataFrame(data=picture_data, columns=columns)

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495965 entries, 0 to 495964
Data columns (total 7 columns):
id          495965 non-null int64
title       495965 non-null object
owner       495965 non-null object
secret      495965 non-null object
farm        495965 non-null int64
server      495965 non-null int64
ispublic    495965 non-null int64
dtypes: int64(4), object(3)
memory usage: 26.5+ MB


In [118]:
df.head()

,id,title,owner,secret,farm,server,ispublic
0,27797327313,Mr. Tanaka｜田中先生,67186028@N06,677ebd93b4,9,8700,1
1,27774358924,The Church on the Hill,16946718@N05,eed93a7b61,9,8769,1
2,27796175404,N160718_022.jpg,61301115@N03,ba41305f68,9,8273,1
3,28130293950,DP2M0766_edited-1.jpg,142536946@N03,12e891a9bf,8,7622,1
4,27797281933,kale,145189424@N02,d1189a576b,8,7470,1


NameError: name 'farm_id' is not defined

### See if Improved Meta Data Works

In [4]:
df = pd.read_csv('../data/tables/BUILDING/flickr_image_search_for_BUILDING_100.csv', sep='|')

/Users/RedQueen/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df.head()

,height_o,originalformat,o_height,owner,id,datetakengranularity,views,datetaken,isfriend,secret,...,farm,dateupload,width_o,originalsecret,license,datetakenunknown,ispublic,server,url_o,title
0,2976,jpg,Nan,13214836@N03,27779631671,0,479,2016-06-23 07:45:30,0,4369778bc4,...,8,1466684145,3968,dcc1c3cee0,1,0,1,7457,https://farm8.staticflickr.com/7457/2777963167...,IMG_20160623_074528
1,4016,jpg,Nan,132343251@N06,27245793504,0,1059,2016-06-19 16:59:41,0,9121d3eb79,...,8,1466693404,6016,bcb7f96fda,1,0,1,7506,https://farm8.staticflickr.com/7506/2724579350...,_DSC9718
2,5184,jpg,Nan,139724231@N02,26090945691,0,70,2016-03-19 15:45:36,0,ee1e7718bd,...,2,1459452771,3456,ccdd619a8c,1,0,1,1683,https://farm2.staticflickr.com/1683/2609094569...,NaN
3,2132,jpg,2132,7676405@N04,6766180221,0,9533,2011-07-04 05:37:34,0,42e507830d,...,8,1458481843,4197,f23564145d,1,0,1,7020,https://farm8.staticflickr.com/7020/6766180221...,Southern Spot...
4,2380,jpg,2380,7676405@N04,11833401363,0,6761,2013-12-12 13:49:33,0,8ce69f6e8c,...,4,1456756820,4230,144a1d1e8a,1,0,1,3736,https://farm4.staticflickr.com/3736/1183340136...,Napoleon's Court...


In [21]:
df['height_o'][-100:][-1:].values[0]

2730

In [13]:
df.applymap(np.isreal)

,height_o,originalformat,o_height,owner,id,datetakengranularity,views,datetaken,isfriend,secret,...,farm,dateupload,width_o,originalsecret,license,datetakenunknown,ispublic,server,url_o,title
0,False,False,False,False,True,True,True,False,True,False,...,True,True,False,False,True,True,True,True,False,False
1,False,False,False,False,True,True,True,False,True,False,...,True,True,False,False,True,True,True,True,False,False
2,False,False,False,False,True,True,True,False,True,False,...,True,True,False,False,True,True,True,True,False,True
3,False,False,False,False,True,True,True,False,True,False,...,True,True,False,False,True,True,True,True,False,False
4,False,False,False,False,True,True,True,False,True,False,...,True,True,False,False,True,True,True,True,False,False
5,False,False,False,False,True,True,True,False,True,False,...,True,True,False,False,True,True,True,True,False,False
6,False,False,False,False,True,True,True,False,True,False,...,True,True,False,False,True,True,True,True,False,False
7,False,False,False,False,True,True,True,False,True,False,...,True,True,False,False,True,True,True,True,False,False
8,False,False,False,False,True,True,True,False,True,False,...,True,True,False,False,True,True,True,True,False,False
9,False,False,False,False,True,True,True,False,True,False,...,True,True,False,False,True,True,True,True,False,False


In [24]:
sum([2,3,6])

11

In [26]:
sum([True, True, 2])

4

In [30]:
np.append(np.array([1,4,7,2,3]),np.array([1,1,1,1]))

array([1, 4, 7, 2, 3, 1, 1, 1, 1])

In [31]:
np.append(np.array([1,4,7,2,3]),1)

array([1, 4, 7, 2, 3, 1])

In [34]:
np.append(np.empty((1,0)),np.array([1,1,1,1]))

array([ 1.,  1.,  1.,  1.])

In [35]:
np.append(np.empty((1,0)),1)

array([ 1.])

In [40]:
np.concatenate((np.array([-8]), np.array([9,1,7,1]), np.array([1,3,1])), axis=0)

array([-8,  9,  1,  7,  1,  1,  3,  1])

In [117]:
np.append(np.array([[9,1,7,1]]), np.array([[1,3,-5,1]]), axis=0)

array([[ 9,  1,  7,  1],
       [ 1,  3, -5,  1]])

In [119]:
np.concatenate((np.array([[9,1,7,1]]), np.array([[1,3,-5,1]])), axis=0)

array([[ 9,  1,  7,  1],
       [ 1,  3, -5,  1]])

In [45]:
from skimage.exposure import histogram as sk_hist

In [46]:
sk_hist(np.array([[1,2,3], [3,2,1]]))

(array([2, 2, 2]), array([1, 2, 3]))

In [114]:
def get_custom_hist(array, lower, upper, nbins):
    """
    Get a histographic distribution of the array.

    array | A numpy array of data.
    lower | Lowest allowable value in histogram results (integer).
    upper | Highest allowable value in histogram results (integer).
    bin_size | Number of bins into which to split the data.
    """
    data = np.round(array.flatten(), decimals=0).astype(float)
    bin_width = ((upper+1) - lower)/float(nbins)
    steps = np.arange(lower, upper+1+bin_width, bin_width)
    counts = np.zeros((len(steps) - 1, ))
    for i, val in enumerate(steps[0:-1]):
        counts[i] = len(data[(data >= steps[i]) & (data < steps[i+1])])
    return counts[0:nbins+1]

In [115]:
array = np.array([[1,4,2,6,8],[6,-8,-2,2,0]])

In [116]:
get_custom_hist(array, -10, 10, 5)

array([ 1.,  1.,  4.,  3.,  1.])